In [1]:
%matplotlib inline

import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import calendar

plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [13]:
from sklearn import linear_model
print(__doc__)

Automatically created module for IPython interactive environment


In [2]:
trip = pd.read_csv('data/trip_train.csv', low_memory=False)
trip['start_date'] = pd.to_datetime(trip['start_date'])
trip['start_hour'] = pd.to_datetime(trip['start_date'], format = '%H', errors = 'coerce').dt.hour
trip['dia'] = trip['start_date'].dt.strftime('%w')
trip['date'] = trip.start_date.dt.date

In [9]:
stations = pd.read_csv('./data/station.csv', low_memory=False)

In [4]:
salida = pd.read_csv('data/trip_test.csv', low_memory=False)
#esto tarda
salida['start_date'] = pd.to_datetime(salida['start_date'])
salida['start_hour'] = pd.to_datetime(salida['start_date'], format = '%H', errors = 'coerce').dt.hour
salida['date'] = salida['start_date'].dt.strftime('%w')


In [89]:
#Distancias Mangatan.
stationsStart = stations
stationsStart['start_station_id'] = stations['id']
stationsStart['start_lat'] = stations['lat']
stationsStart['start_long'] = stations['long']
stationsStart = stationsStart.loc[:,['start_station_id','start_lat','start_long']]
stationsEnd = stations
stationsEnd['end_station_id'] = stations['id']
stationsEnd['end_lat'] = stations['lat']
stationsEnd['end_long'] = stations['long']
stationsEnd = stationsEnd.loc[:,['end_station_id','end_lat','end_long']]
tripEntrada = trip
tripEntrada = pd.merge(tripEntrada, stationsStart, on='start_station_id', how='right')
tripEntrada = pd.merge(tripEntrada, stationsEnd, on='end_station_id', how='right')
tripEntrada['distancia'] = (((tripEntrada['start_lat']-tripEntrada['end_lat'])**2+(tripEntrada['start_long']-tripEntrada['end_long'])**2)**0.5)*10000
tripsSalida = salida
tripsSalida = pd.merge(tripsSalida, stationsStart, on='start_station_id', how='right')
tripsSalida = pd.merge(tripsSalida, stationsEnd, on='end_station_id', how='right')
tripsSalida['distancia'] = (((tripsSalida['start_lat']-tripsSalida['end_lat'])**2+(tripsSalida['start_long']-tripsSalida['end_long'])**2)**0.5)*10000


In [90]:
salidaFinal = trip[trip.date == str(7)]
salidaFinal = salidaFinal.loc[:,['id','duration']]

#tripEntrada = tripEntrada[tripEntrada.duration > 60*2]     
tripEntrada = tripEntrada[tripEntrada.distancia != 0] 
tripEntrada = tripEntrada[tripEntrada.duration < 60*60*24]  

#import math
#x=float('nan')
#math.isnan(x)

T = 0;
for y in range(0, 70):
    for x in range(0, 7):
        for z in range (0,25):
            tripAuxX = tripEntrada[tripEntrada.start_station_id == stations2.id[y]]
            tripsSalidaX = tripsSalida[tripsSalida.start_station_id == stations2.id[y]]         
            tripsAuxX = tripAuxX[tripAuxX.start_hour == z]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.start_hour == z]            
            tripAuxX = tripAuxX[tripAuxX.dia == str(x)]
            tripsSalidaX = tripsSalidaX[tripsSalidaX.date == str(x)]
        
            if(tripAuxX.count().id > 0):
                entradaX = tripAuxX.loc[:,['distancia']].as_matrix(columns=None)
                entradaY = tripAuxX.loc[:,['duration']].as_matrix(columns=None)

                regr = linear_model.LinearRegression()
                regr.fit(entradaY, entradaY)

                if(tripsSalidaX.count().id > 0):
                    salidaX = tripsSalidaX.loc[:,['distancia']].as_matrix(columns=None)
                    salidaY = regr.predict(salidaX) 
                    tripsSalidaX['duration'] = salidaY

                    tripsX= tripsSalidaX.loc[:,['id','duration']]
                    salidaFinal = salidaFinal.append(tripsX)

In [91]:
menores = salidaFinal[salidaFinal.duration <60]      
menores['duration'] = 60
salidaFinal2 = salidaFinal[salidaFinal.duration >60] 
salidaFinal2 = salidaFinal2.append(menores)

/home/geco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [92]:
salidaFinal2 = salidaFinal2.sort_values(by='id', ascending=True)
salidaFinal2.to_csv("salida.csv", index=False)

In [93]:
#119998 
salidaFinal2.count()

id          119998
duration    119998
dtype: int64

In [77]:
tripAuxX.count().id

1446